In [23]:
pip install mne

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 42.3 MB/s eta 0:00:00


In [1]:


import pandas as pd
import numpy as np

import tensorflow as tf
from keras import backend as K
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout
from keras.layers import BatchNormalization
from keras.regularizers import L1L2
from keras._tf_keras.keras.layers import Conv2D, BatchNormalization, RepeatVector,MaxPooling2D, Flatten, Dense, Dropout,GlobalAveragePooling2D,LSTM, TimeDistributed, Reshape,LayerNormalization, MultiHeadAttention, Dense, GlobalAveragePooling1D, Input, RandomFlip, RandomRotation, RandomZoom, RandomZoom, RandomTranslation, RandomContrast, RandomBrightness,Conv3D, BatchNormalization, MaxPooling3D,  Conv2D, MaxPooling2D, Dropout, Reshape, LSTM, Dense, LayerNormalization, MultiHeadAttention, Input, Add
from sklearn.model_selection import train_test_split
from keras._tf_keras.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau


In [24]:
import scipy.io
import numpy as np
import glob
import os
import numpy as np
from scipy.signal import spectrogram
import cv2 #pip install opencv-python
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, SpectralClustering, DBSCAN, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
import numpy as np
from tensorflow.keras.utils import to_categorical
from keras._tf_keras.keras.applications import MobileNetV2
from keras._tf_keras.keras.utils import plot_model
from keras._tf_keras.keras.saving import register_keras_serializable
from keras._tf_keras.keras.regularizers import l2 as l2_reg
from keras._tf_keras.keras.preprocessing.image import ImageDataGenerator
from keras._tf_keras.keras.models import load_model
from keras._tf_keras.keras import Model, optimizers, Input, Layer
from keras._tf_keras.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from keras._tf_keras.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix
import tensorflow as tf
import shap
import pandas as pd
import seaborn as sns
import mne
import numpy as np
import mne
from mne.preprocessing import ICA
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import tensorflow as tf
from keras._tf_keras.keras.models import Sequential
from keras._tf_keras.keras.layers import Conv2D, BatchNormalization, RepeatVector,MaxPooling2D, Flatten, Dense, Dropout,GlobalAveragePooling2D,LSTM, TimeDistributed, Reshape,LayerNormalization, MultiHeadAttention, Dense, GlobalAveragePooling1D, Input, RandomFlip, RandomRotation, RandomZoom, RandomZoom, RandomTranslation, RandomContrast, RandomBrightness,Conv3D, BatchNormalization, MaxPooling3D
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
base_folder = "/content/drive/MyDrive/"
eeg_data,label=load_eeg_data_from_groups(base_folder)

In [2]:

ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

<html><h1><p style="color:red">Loading Data / Making Data </p></h1></html>

In [3]:

d = '/content/drive/MyDrive/UCI_HAR_Dataset'

SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
    ]

In [4]:

def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)


def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'{d}/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).to_numpy()
        )

    return np.transpose(signals_data, (1, 2, 0))

In [5]:
def load_y(subset):

    filename = f'{d}/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).to_numpy()

In [6]:
def load_data():

    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [8]:

def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [9]:

X_train, X_test, Y_train, Y_test = load_data()

<ipython-input-4-db863a557e05>:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  return pd.read_csv(filename, delim_whitespace=True, header=None)
<ipython-input-4-db863a557e05>:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  return pd.read_csv(filename, delim_whitespace=True, header=None)
<ipython-input-4-db863a557e05>:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  return pd.read_csv(filename, delim_whitespace=True, header=None)
<ipython-input-4-db863a557e05>:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  return pd.read_csv(filename, delim_whitespace=True, header=None)
<ipython-input-4-db863a557e0

In [22]:
print(X_train.shape,X_test.shape)

(7352, 128, 9) (2947, 128, 9)


In [24]:
s=X_train.shape

In [27]:
X_train_r=np.expand_dims(X_train,axis=3)
X_test_r=np.expand_dims(X_test,axis=3)

In [28]:
print(X_train_r.shape,X_test_r.shape)

(7352, 128, 9, 1) (2947, 128, 9, 1)


<html><h1><p style="color:red">1. Defining the Architecture of 1-Layer of LSTM </p></h1></html>

In [32]:
input_shape=X_train_r.shape[1:]
num_classes=6

In [70]:
model = Sequential()

model.add(Conv2D(16, (2, 1),
          strides=(2, 1),
          activation='relu',
          input_shape=input_shape))
model.add(MaxPooling2D((2, 1), strides=(2, 1), padding='same'))


model.add(Dropout(0.3))

model.add(Flatten())

model.add(RepeatVector(4))

model.add(LSTM(128))
model.add(Dropout(0.2))




model.add(Dense(num_classes, activation="softmax"))

model.summary()

Model: "sequential_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_21 (Conv2D)                   │ (None, 64, 9, 16)           │              48 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_18 (MaxPooling2D)      │ (None, 32, 9, 16)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_52 (Dropout)                 │ (None, 32, 9, 16)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_15 (Flatten)                 │ (None, 4608)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ repeat_vector_14 (RepeatVector)      │ (None, 4, 4608)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_25 (LSTM)                       │ (None, 128)                 │       2,425,344 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_53 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_29 (Dense)                     │ (None, 6)                   │             774 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,426,166 (9.26 MB)

 Trainable params: 2,426,166 (9.26 MB)

 Non-trainable params: 0 (0.00 B)

In [80]:
model = Sequential()


model.add(Conv2D(16, (2, 2), strides=(2, 2), activation='relu', input_shape=(128, 9, 1)))
model.add(MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
model.add(Dropout(0.3))

new_timesteps = 32
new_features = 2 * 16
model.add(Reshape((new_timesteps, new_features)))


model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(64))
model.add(Dropout(0.2))

model.add(Dense(16, activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(num_classes, activation="softmax"))

model.summary()


Model: "sequential_23"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_26 (Conv2D)                   │ (None, 64, 4, 16)           │              80 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_23 (MaxPooling2D)      │ (None, 32, 2, 16)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_58 (Dropout)                 │ (None, 32, 2, 16)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 32, 32)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_26 (LSTM)                       │ (None, 32, 128)             │          82,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_59 (Dropout)                 │ (None, 32, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_27 (LSTM)                       │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_60 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_30 (Dense)                     │ (None, 16)                  │           1,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_61 (Dropout)                 │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_31 (Dense)                     │ (None, 6)                   │             102 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 133,062 (519.77 KB)

 Trainable params: 133,062 (519.77 KB)

 Non-trainable params: 0 (0.00 B)

In [85]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)
mc = tf.keras.callbacks.ModelCheckpoint('best_model_emot.keras', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6, verbose=1)

model.compile(optimizer ="adam", loss ='categorical_crossentropy', metrics=['accuracy'])
m=model.fit(X_train, Y_train,epochs=30,batch_size=32,verbose=1,validation_data=(X_test, Y_test), callbacks =[mc,reduce_lr])

Epoch 1/30
228/230 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9357 - loss: 0.1731
Epoch 1: val_accuracy improved from -inf to 0.89447, saving model to best_model_emot.keras
230/230 ━━━━━━━━━━━━━━━━━━━━ 56s 13ms/step - accuracy: 0.9357 - loss: 0.1730 - val_accuracy: 0.8945 - val_loss: 0.2754 - learning_rate: 0.0010
Epoch 2/30
230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9382 - loss: 0.1648
Epoch 2: val_accuracy improved from 0.89447 to 0.89786, saving model to best_model_emot.keras
230/230 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9382 - loss: 0.1647 - val_accuracy: 0.8979 - val_loss: 0.3994 - learning_rate: 0.0010
Epoch 3/30
230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9401 - loss: 0.1647
Epoch 3: val_accuracy did not improve from 0.89786
230/230 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9402 - loss: 0.1646 - val_accuracy: 0.8945 - val_loss: 0.3200 - learning_rate: 0.0010
Epoch 4/30
226/230 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9423 - loss

In [87]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report, confusion_matrix


y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)


y_true = np.argmax(Y_test, axis=1)

93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step


In [88]:

accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy:.4f}")


recall = recall_score(y_true, y_pred, average='weighted')
print(f"Recall: {recall:.4f}")


precision = precision_score(y_true, y_pred, average='weighted')
print(f"Precision: {precision:.4f}")


f1 = f1_score(y_true, y_pred, average='weighted')
print(f"F1-score: {f1:.4f}")


print("\nClassification Report:")
print(classification_report(y_true, y_pred))


print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))

Accuracy: 0.9301
Recall: 0.9301
Precision: 0.9302
F1-score: 0.9300

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.95      0.95       496
           1       0.97      0.93      0.95       471
           2       0.92      0.97      0.94       420
           3       0.87      0.86      0.86       491
           4       0.88      0.87      0.87       532
           5       1.00      1.00      1.00       537

    accuracy                           0.93      2947
   macro avg       0.93      0.93      0.93      2947
weighted avg       0.93      0.93      0.93      2947


Confusion Matrix:
[[473   0  23   0   0   0]
 [ 21 439  11   0   0   0]
 [  4  10 406   0   0   0]
 [  1   3   0 421  66   0]
 [  0   2   0  65 465   0]
 [  0   0   0   0   0 537]]


In [93]:
input_layer = Input(shape=(128, 9, 1))

x = Conv2D(16, (2, 2), strides=(2, 2), activation='relu')(input_layer)
x = MaxPooling2D((2, 2), strides=(2, 2), padding='same')(x)
x = Dropout(0.3)(x)


new_timesteps = 32
new_features = 2 * 16

x = Reshape((new_timesteps, new_features))(x)


x = LSTM(128, return_sequences=True)(x)
x = Dropout(0.2)(x)

x = LSTM(64, return_sequences=True)(x)
x = Dropout(0.2)(x)


def TransformerEncoder(x, e_dim, num, ff_dim, rate=0.1):
    attn_output = MultiHeadAttention(num_heads=num, key_dim=e_dim)(x, x)
    attn_output = Dropout(rate)(attn_output)
    out1 = LayerNormalization(epsilon=1e-6)(x + attn_output)

    ffn_output = Dense(ff_dim, activation="relu")(out1)
    ffn_output = Dense(e_dim)(ffn_output)
    ffn_output = Dropout(rate)(ffn_output)

    return LayerNormalization(epsilon=1e-6)(out1 + ffn_output)

x = TransformerEncoder(x, e_dim=64, num=4, ff_dim=128)
x = GlobalAveragePooling1D()(x)


x = Dense(16, activation="relu")(x)
x = Dropout(0.2)(x)

x = Dense(num_classes, activation="softmax")(x)


model = Model(inputs=input_layer, outputs=x)

model.summary()


Model: "functional_213"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_26            │ (None, 128, 9, 1)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_29 (Conv2D)        │ (None, 64, 4, 16)      │             80 │ input_layer_26[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_26          │ (None, 32, 2, 16)      │              0 │ conv2d_29[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_74 (Dropout)      │ (None, 32, 2, 16)      │              0 │ max_pooling2d_26[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_3 (Reshape)       │ (None, 32, 32)         │              0 │ dropout_74[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_32 (LSTM)            │ (None, 32, 128)        │         82,432 │ reshape_3[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_75 (Dropout)      │ (None, 32, 128)        │              0 │ lstm_32[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_33 (LSTM)            │ (None, 32, 64)         │         49,408 │ dropout_75[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_76 (Dropout)      │ (None, 32, 64)         │              0 │ lstm_33[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_2    │ (None, 32, 64)         │         66,368 │ dropout_76[0][0],      │
│ (MultiHeadAttention)      │                        │                │ dropout_76[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_78 (Dropout)      │ (None, 32, 64)         │              0 │ multi_head_attention_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_2 (Add)               │ (None, 32, 64)         │              0 │ dropout_76[0][0],      │
│                           │                        │                │ dropout_78[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_4     │ (None, 32, 64)         │            128 │ add_2[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_38 (Dense)          │ (None, 32, 128)        │          8,320 │ layer_normalization_4… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_39 (Dense)          │ (None, 32, 64)         │          8,256 │ dense_38[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_79 (Dropout)      │ (None, 32, 64)         │              0 │ dense_39[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_3 (Add)               │ (None, 32, 64)         │              0 │ layer_normalization_4… │
│                      

 Total params: 216,262 (844.77 KB)

 Trainable params: 216,262 (844.77 KB)

 Non-trainable params: 0 (0.00 B)

In [94]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)
mc = tf.keras.callbacks.ModelCheckpoint('best_model_emot.keras', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6, verbose=1)

model.compile(optimizer ="adam", loss ='categorical_crossentropy', metrics=['accuracy'])
m=model.fit(X_train, Y_train,epochs=30,batch_size=32,verbose=1,validation_data=(X_test, Y_test), callbacks =[mc,reduce_lr])

Epoch 1/30
230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.4529 - loss: 1.2494
Epoch 1: val_accuracy improved from -inf to 0.70682, saving model to best_model_emot.keras
230/230 ━━━━━━━━━━━━━━━━━━━━ 12s 27ms/step - accuracy: 0.4534 - loss: 1.2481 - val_accuracy: 0.7068 - val_loss: 0.6290 - learning_rate: 0.0010
Epoch 2/30
228/230 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7178 - loss: 0.6344
Epoch 2: val_accuracy improved from 0.70682 to 0.77944, saving model to best_model_emot.keras
230/230 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.7182 - loss: 0.6336 - val_accuracy: 0.7794 - val_loss: 0.5391 - learning_rate: 0.0010
Epoch 3/30
230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8066 - loss: 0.4783
Epoch 3: val_accuracy improved from 0.77944 to 0.83101, saving model to best_model_emot.keras
230/230 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.8067 - loss: 0.4782 - val_accuracy: 0.8310 - val_loss: 0.4260 - learning_rate: 0.0010
Epoch 4/30
227/230 ━━━━━━━━━━━━━━

In [95]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)
mc = tf.keras.callbacks.ModelCheckpoint('best_model_emot.keras', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6, verbose=1)

model.compile(optimizer ="adam", loss ='categorical_crossentropy', metrics=['accuracy'])
m=model.fit(X_train, Y_train,epochs=50,batch_size=32,verbose=1,validation_data=(X_test, Y_test), callbacks =[mc,reduce_lr])

Epoch 1/50
230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9446 - loss: 0.1521
Epoch 1: val_accuracy improved from -inf to 0.91551, saving model to best_model_emot.keras
230/230 ━━━━━━━━━━━━━━━━━━━━ 11s 20ms/step - accuracy: 0.9446 - loss: 0.1521 - val_accuracy: 0.9155 - val_loss: 0.2580 - learning_rate: 0.0010
Epoch 2/50
228/230 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9518 - loss: 0.1262
Epoch 2: val_accuracy did not improve from 0.91551
230/230 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.9518 - loss: 0.1262 - val_accuracy: 0.9097 - val_loss: 0.3377 - learning_rate: 0.0010
Epoch 3/50
230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9504 - loss: 0.1351
Epoch 3: val_accuracy improved from 0.91551 to 0.92195, saving model to best_model_emot.keras
230/230 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.9504 - loss: 0.1351 - val_accuracy: 0.9220 - val_loss: 0.2546 - learning_rate: 0.0010
Epoch 4/50
228/230 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9493 - lo

In [96]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report, confusion_matrix


y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)


y_true = np.argmax(Y_test, axis=1)

93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step


In [97]:

accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy:.4f}")


recall = recall_score(y_true, y_pred, average='weighted')
print(f"Recall: {recall:.4f}")


precision = precision_score(y_true, y_pred, average='weighted')
print(f"Precision: {precision:.4f}")


f1 = f1_score(y_true, y_pred, average='weighted')
print(f"F1-score: {f1:.4f}")

print("\nClassification Report:")
print(classification_report(y_true, y_pred))


print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))

Accuracy: 0.9430
Recall: 0.9430
Precision: 0.9431
F1-score: 0.9429

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.97      0.98       496
           1       0.98      0.97      0.97       471
           2       0.96      0.98      0.97       420
           3       0.88      0.84      0.86       491
           4       0.86      0.89      0.88       532
           5       1.00      1.00      1.00       537

    accuracy                           0.94      2947
   macro avg       0.94      0.94      0.94      2947
weighted avg       0.94      0.94      0.94      2947


Confusion Matrix:
[[482   1  13   0   0   0]
 [  7 458   6   0   0   0]
 [  1   7 412   0   0   0]
 [  0   2   0 414  75   0]
 [  0   1   0  55 476   0]
 [  0   0   0   0   0 537]]


In [11]:
pip install tsfel

In [106]:
import tsfel
import pandas as pd
import numpy as np


cfg = tsfel.get_features_by_domain("statistical")

X_train_list = [pd.DataFrame(X_train[i]) for i in range(len(X_train))]
X_test_list = [pd.DataFrame(X_test[i]) for i in range(len(X_test))]

X_train_tsfel = pd.DataFrame([tsfel.time_series_features_extractor(cfg, x, n_jobs=-1) for x in X_train_list])
X_test_tsfel = pd.DataFrame([tsfel.time_series_features_extractor(cfg, x, n_jobs=-1) for x in X_test_list])

print("Fixed X_train_tsfel shape:", X_train_tsfel.shape)
print("Y_train shape:", Y_train.shape)

<ipython-input-106-a30ee9122ca3>:13: UserWarning: Using default sampling frequency set in configuration file.
  X_train_tsfel = pd.DataFrame([tsfel.time_series_features_extractor(cfg, x, n_jobs=-1) for x in X_train_list])


<ipython-input-106-a30ee9122ca3>:13: UserWarning: Using default sampling frequency set in configuration file.
  X_train_tsfel = pd.DataFrame([tsfel.time_series_features_extractor(cfg, x, n_jobs=-1) for x in X_train_list])


<ipython-input-106-a30ee9122ca3>:13: UserWarning: Using default sampling frequency set in configuration file.
  X_train_tsfel = pd.DataFrame([tsfel.time_series_features_extractor(cfg, x, n_jobs=-1) for x in X_train_list])


<ipython-input-106-a30ee9122ca3>:13: UserWarning: Using default sampling frequency set in configuration file.
  X_train_tsfel = pd.DataFrame([tsfel.time_series_features_extractor(cfg, x, n_jobs=-1) for x in X_train_list])


<ipython-input-106-a30ee9122ca3>:13: UserWarning: Using default sampling frequency set in configuration file.
  X_train_tsfel = pd.DataFrame([tsfel.time_series_features_extractor(cfg, x, n_jobs=-1) for x in X_train_list])


...

<ipython-input-106-a30ee9122ca3>:13: UserWarning: Using default sampling frequency set in configuration file.
  X_train_tsfel = pd.DataFrame([tsfel.time_series_features_extractor(cfg, x, n_jobs=-1) for x in X_train_list])


<ipython-input-106-a30ee9122ca3>:13: UserWarning: Using default sampling frequency set in configuration file.
  X_train_tsfel = pd.DataFrame([tsfel.time_series_features_extractor(cfg, x, n_jobs=-1) for x in X_train_list])


<ipython-input-106-a30ee9122ca3>:13: UserWarning: Using default sampling frequency set in configuration file.
  X_train_tsfel = pd.DataFrame([tsfel.time_series_features_extractor(cfg, x, n_jobs=-1) for x in X_train_list])


<ipython-input-106-a30ee9122ca3>:13: UserWarning: Using default sampling frequency set in configuration file.
  X_train_tsfel = pd.DataFrame([tsfel.time_series_features_extractor(cfg, x, n_jobs=-1) for x in X_train_list])


<ipython-input-106-a30ee9122ca3>:13: UserWarning: Using default sampling frequency set in configuration file.
  X_train_tsfel = pd.DataFrame([tsfel.time_series_features_extractor(cfg, x, n_jobs=-1) for x in X_train_list])


In [21]:
import multiprocessing
multiprocessing.cpu_count()


2

In [14]:
X_train.shape

(7352, 128, 9)

In [ ]:
#Due to time constraints and ongoing mid-semester exams unable to fully implement tsfel.